TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

## Training for new target(angle)

In [1]:
input_width = 160
input_height = 100
channels = 3
rotations_count = 2
data_path = "D:\\Python\\Keras\\Wormax\\data_prepared\\"
model_name = 'models/worm_single_regr_5.h5'
look_forward = 0
use_rotation = True
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models, optimizers
import keras.backend as K

# Radians, please
def mean_angle_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs(tf.atan2(tf.sin(y_true - y_pred), tf.cos(y_true - y_pred))))

def define_model():
    model = models.Sequential()
        
    model.add(layers.Convolution2D(32, (8, 8), strides=(2, 2), activation='relu',
                            input_shape=(input_height, input_width, channels)))
    model.add(layers.Convolution2D(32, (8, 8), activation='relu'))
    model.add(layers.Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(layers.Convolution2D(64, (4, 4), activation='relu'))
    model.add(layers.Convolution2D(64, (3, 3), activation='relu'))
    model.add(layers.Convolution2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))    
           
    model.compile(optimizer=optimizers.Adam(lr=5e-4),
                  loss=mean_angle_error)
    
    model.summary()
    return model

In [3]:
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 47, 77, 32)        6176      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 70, 32)        65568     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 34, 64)        32832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 31, 64)        65600     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 29, 64)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 27, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 20736)             0         
__________

In [4]:
# little prepocessing
from math import atan2, pi
import cv2

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

# Rotating frame and direction counter-clockwise
def get_rotated_frame(img, rotation, target_angle, rotations_count):
    if rotation != 0:
        rows, cols = len(img),len(img[0])
        rot_M = cv2.getRotationMatrix2D((cols/2, rows/2), rotation*360/rotations_count, 1)
        img = cv2.warpAffine(img, rot_M, (cols, rows))
        target_angle = (target_angle + rotation/rotations_count*2*pi)%(2*pi)
    return img, target_angle

In [5]:
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, rotations_count, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        arr = np.load(data_dir + listdir[file_i])        
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        # Expanding blocks
        data = []        
        for i in arr:
            for j in i:
                data.append(j)
        data = np.array(data)

        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        # Generating y
        data_targets = np.zeros((len(data)-look_forward))
        for i in range(len(data_targets)):
            data_targets[i] = get_angle(*data[i+look_forward][1][:2])
            #data_targets[i] = np.array(to_categorical(get_direction(*data[i+look_forward][1][:2]), num_classes=num_classes))
        data = data[:len(data)-look_forward]
        
        # Only X
        data_features = np.zeros((len(data), input_height, input_width, channels))
        for i in range(len(data)):
            data_features[i] = data[i][0][0]
        
        indexes = np.arange(len(data_features)-look_forward)        
        if shuffle:
            np.random.shuffle(indexes)
        
        # Rotating cycle(no rotation for validation)
        rotate_times = rotations_count if role == 'train' and use_rotation else 1
        for rot_i in range(rotate_times):
            # Batches forming cycle
            for i in range(0, len(data)-look_forward-batch_size, batch_size):
                samples = data_features.take(indexes[i:i+batch_size], axis=0)
                targets = data_targets.take(indexes[i:i+batch_size], axis=0)
                samples_rot = np.zeros_like(samples)
                targets_rot = np.zeros((batch_size))
                
                # Rotating
                for j in range(batch_size):
                    if use_rotation:
                        rotation = (rot_i+j+i)%rotations_count
                        img, target = get_rotated_frame(samples[j], rotation, targets[j], rotations_count)
                    else:
                        img, target = samples[j], targets[j]
                    samples_rot[j] = img
                    targets_rot[j] = target
                    
                # Normilize
                samples_rot = samples_rot / 255
                yield samples_rot, targets_rot

In [6]:
import numpy as np

train_generator = generator(data_path, rotations_count, 'train', batch_size=16)
validation_generator = generator(data_path, rotations_count, 'validation', batch_size=32)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(16, 100, 160, 3)
(16,)


In [7]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 100:
            break
    print(classes)

### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

# TODO
    1) make shuffled rotations
    2) tune look_forward
    3) change target to angle

In [ ]:
initial_epoch = 0
if not True:
    model.load_weights(model_name)
    initial_epoch = 176

In [ ]:
'''class_weight = {0: 1.15,
                 1: 1.13,
                 2: 1.27,
                 3: 1.51,
                 4: 1.46,
                 5: 1.19,
                 6: 1.0,
                 7: 1.04,
                 8: 1.11,
                 9: 1.2,
                 10: 1.27,
                 11: 1.34}'''

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor='val_loss',
        save_best_only=True,
    )
    
]
'''keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.5,                              
        patience=70, 
        min_lr=0.00001
    )'''

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            #class_weight=class_weight,
                            callbacks=callbacks,
                            initial_epoch=initial_epoch
                        )

Epoch 1/500
100/100 [==============================] - 16s 155ms/step - loss: 1.5300 - val_loss: 1.4565
Epoch 2/500
100/100 [==============================] - 6s 61ms/step - loss: 1.3580 - val_loss: 1.3169
Epoch 3/500
100/100 [==============================] - 13s 131ms/step - loss: 1.2049 - val_loss: 1.2050
Epoch 4/500
100/100 [==============================] - 7s 65ms/step - loss: 1.1217 - val_loss: 1.0487
Epoch 5/500
100/100 [==============================] - 10s 99ms/step - loss: 1.0014 - val_loss: 1.0349
Epoch 6/500
100/100 [==============================] - 6s 62ms/step - loss: 0.9978 - val_loss: 0.9460
Epoch 7/500
100/100 [==============================] - 13s 135ms/step - loss: 0.9688 - val_loss: 1.0029
Epoch 8/500
100/100 [==============================] - 13s 132ms/step - loss: 0.8870 - val_loss: 0.8492
Epoch 9/500
100/100 [==============================] - 11s 115ms/step - loss: 0.8077 - val_loss: 0.8609
Epoch 10/500
100/100 [==============================] - 7s 68ms/step - 

Epoch 80/500
100/100 [==============================] - 14s 137ms/step - loss: 0.6431 - val_loss: 0.5154
Epoch 81/500
100/100 [==============================] - 6s 61ms/step - loss: 0.6415 - val_loss: 0.5332
Epoch 82/500
100/100 [==============================] - 16s 163ms/step - loss: 0.6345 - val_loss: 0.5963
Epoch 83/500
100/100 [==============================] - 7s 67ms/step - loss: 0.6107 - val_loss: 0.6417
Epoch 84/500
100/100 [==============================] - 12s 120ms/step - loss: 0.6044 - val_loss: 0.6219
Epoch 85/500
100/100 [==============================] - 6s 62ms/step - loss: 0.6982 - val_loss: 0.7092
Epoch 86/500
100/100 [==============================] - 9s 94ms/step - loss: 0.6756 - val_loss: 0.7374
Epoch 87/500
100/100 [==============================] - 24s 241ms/step - loss: 0.6443 - val_loss: 0.7368
Epoch 88/500
100/100 [==============================] - 6s 65ms/step - loss: 0.7463 - val_loss: 0.9368
Epoch 89/500
100/100 [==============================] - 15s 147ms

100/100 [==============================] - 6s 62ms/step - loss: 0.4867 - val_loss: 0.6037
Epoch 159/500
100/100 [==============================] - 23s 228ms/step - loss: 0.5343 - val_loss: 0.6402
Epoch 160/500
100/100 [==============================] - 14s 141ms/step - loss: 0.5635 - val_loss: 0.6568
Epoch 161/500
100/100 [==============================] - 7s 65ms/step - loss: 0.5745 - val_loss: 0.7977
Epoch 162/500
100/100 [==============================] - 14s 141ms/step - loss: 0.5480 - val_loss: 0.7680
Epoch 163/500
100/100 [==============================] - 7s 66ms/step - loss: 0.5578 - val_loss: 0.5581
Epoch 164/500
100/100 [==============================] - 14s 142ms/step - loss: 0.5417 - val_loss: 0.5848
Epoch 165/500
100/100 [==============================] - 6s 65ms/step - loss: 0.5435 - val_loss: 0.7124
Epoch 166/500
100/100 [==============================] - 19s 188ms/step - loss: 0.5387 - val_loss: 0.6452
Epoch 167/500
100/100 [==============================] - 6s 63ms/ste

In [ ]:
from keras import backend as K
K.clear_session()